<a href="https://colab.research.google.com/github/marinobeach-cloud/textClassificationSMS/blob/main/fcc_smsTextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["label", "message"])

# Explorar los datos
print(train_df.head())
print(f"\nTrain shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nLabel distribution:\n{train_df['label'].value_counts()}")

# Convertir labels a valores numéricos (ham=0, spam=1)
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

train_messages = train_df['message'].values
train_labels = train_df['label'].values
test_messages = test_df['message'].values
test_labels = test_df['label'].values

# Vectorizador mejorado
vectorizer = keras.layers.TextVectorization(
    max_tokens=5000,  # Reducido para evitar overfitting
    output_mode='int',
    output_sequence_length=50  # Reducido, los SMS son cortos
)

vectorizer.adapt(train_messages)

# Modelo mejorado
model = keras.Sequential([
    vectorizer,
    keras.layers.Embedding(input_dim=5000, output_dim=32),  # Aumentado embedding
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),  # LSTM bidireccional
    keras.layers.Bidirectional(keras.layers.LSTM(32)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Entrenar con más épocas
history = model.fit(
    train_messages,
    train_labels,
    epochs=20,  # Más épocas para mejor aprendizaje
    batch_size=32,
    validation_data=(test_messages, test_labels),
    verbose=1
)

def predict_message(pred_text):
  input_data = np.array([pred_text], dtype=object)
  prediction_prob = model.predict(input_data, verbose=0)[0][0]

  # Determinar la etiqueta basada en la probabilidad
  label = "spam" if prediction_prob > 0.5 else "ham"

  # Retornar lista con [probabilidad, etiqueta]
  prediction = [float(prediction_prob), label]

  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
